In [42]:
import wget
import os
from selenium import webdriver
from datetime import datetime

from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

In [68]:
PATH = r'C:\Program Files (x86)\chromedriver.exe'
driver = webdriver.Chrome(PATH)

In [69]:
### Warning!!! Will erase all data!!!
import pandas as pd
data = [{'Suburb': 0, 'Address': 0, 'Description': 0, 'Price': 0, 'Amenities': 0, 'Type': 0, 'Date': 0, 'Insights': 0, 'Schools': 0, 'URL': 0}] 
listings = pd.DataFrame(data)
listings.head()

,Suburb,Address,Description,Price,Amenities,Type,Date,Insights,Schools,URL
0,0,0,0,0,0,0,0,0,0,0


In [71]:
timestamp = 1545730073
dt_object = datetime.fromtimestamp(timestamp)

file = open("ErrorLog.txt", "a")

file.write('\nRan at ')
file.write(str(dt_object))

file.close()

In [81]:
### Scrapper in Full

### Debug
add_record_count = 0
time = datetime.now
file = open("ErrorLog.txt", "a")
file2 = open("UrlLog.txt", "a")

#Landing page for melbourne house prices
page_link = "https://www.domain.com.au/auction-results/melbourne/"

for num_1 in range(3,4):
    for num_2 in range(1,2):
        #looping through suburbs
        
        #Going to melbourne landing page
        driver.get(page_link)
        
        #Finding and clicking on the suburb listing
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div/main/section/div/div[2]/div[2]/div/div/div/div[' + str(num_1) + ']/article[' + str(num_2) + ']/div/a[1]')))
            element = driver.find_element_by_xpath('/html/body/div[1]/div/main/section/div/div[2]/div[2]/div/div/div/div[' + str(num_1) + ']/article[' + str(num_2) + ']/div/a[1]')
            
            element.click()
        except:
            break
        
        #Grabbing the name of the suburb
        try:
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div[4]/div[2]/div[1]/div[1]/h1')))
            element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[4]/div[2]/div[1]/div[1]/h1')
            suburb = element.text
        except:
            pass
        
        #Going through pages of listings within suburbs
        try:
            current_url = driver.current_url 
            for page in range(1,2):
                #looping through pages of listings with a suburb
                if num == 1:
                    driver.get(current_url)
                else:
                    try:
                        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div[4]/div[2]/div[1]/div[2]/ul/li[1]/div/div[1]/div[1]/div/div/div/div/div[2]/div/a/div/img')))
                        driver.get(current_url + "?page=" + str(page))
                    except:
                        file.write("\nGoing through pages of listings within suburbs broken at page" + str(page))
                        break

                for listing_num in range(1,11):
                    #looping through listings in a page
                    reset_point = driver.current_url

                    try:
                        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div[4]/div[2]/div[1]/div[2]/ul/li[' + str(listing_num) + ']/div/div[2]/div/a/h2/span[1]')))
                        element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[4]/div[2]/div[1]/div[2]/ul/li[' + str(listing_num) + ']/div/div[2]/div/a/h2/span[1]')
                        element.click()

                        #Grabs listing description
                        try:
                            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//button[@data-testid="listing-details__description-button"]')))
                            driver.find_element_by_xpath('//button[@data-testid="listing-details__description-button"]').send_keys(u'\ue007')
                            WebDriverWait(driver, 1)
                        except:
                            file.write('\nFailed to expand description in url' + str(driver.current_url) + "Page" + str(page) + "listing" + str(listing_num))
                            pass
                        listing_description = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[5]/div/div/div[3]/div[1]/div/div/div').text

                        #Grabs listing's address
                        #element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[2]/div/div[1]')
                        #listing_address = element.text

                        #Grabs listing's price
                        element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[1]/div[1]/div')
                        listing_price = element.text

                        #Grabs listing's Amenities
                        element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[3]/div')
                        listing_amenities = element.text

                        #Grabs listing's property type
                        element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[4]/span')
                        listing_property_type = element.text

                        #Grabs listing's sold date
                        element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/span[2]')
                        listing_sold_date = element.text

                        try:
                            #Grabs listing's domain insight
                            element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[5]/div/div/p')
                            listing_domain_insight = element.text

                            #Grabs School details
                            driver.find_element_by_xpath('//button[@data-testid="fe-co-school-catchment-view-more-link"]').send_keys(u'\ue007')
                            element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[9]/div/div/section/div[3]/ul')
                            listing_schools = element.text
                        except:
                            file.write('\nFailed to expand schools in url' + str(driver.current_url) + "Page" + str(page) + "listing" + str(listing_num))
                            pass

                        #Grabs listing url
                        listing_url = driver.current_url
                        file2.write("\n" + listing_url)

                        #Storing data
                        data = [{'Suburb': suburb,'Address': listing_address, 'Description': listing_description, 'Price': listing_price, 'Amenities': listing_amenities, 'Type': listing_property_type, 'Date': listing_sold_date, 'Insights': listing_domain_insight, 'Schools': listing_schools, 'URL': listing_url}] 
                        output = pd.DataFrame(data)    
                        listings = listings.append(output, ignore_index=True)
                        
                        add_record_count += 1
                        print(add_record)
                    except:
                        file.write("\nError - " + str(listing_num) + " Not found in page" + str(page))
                        pass
                    
                    listings.to_csv("databackup.csv", encoding='utf-8', index=False)

                    driver.get(reset_point)

        except:
            file.write('\nUncaptured Error')
            pass
file.close()
file2.close()

In [80]:
listings.to_csv("databackup.csv", encoding='utf-8', index=False)

In [ ]:
### Scrap Code Backup

for num_page in range(1,26):
    listing_address = listing_description = listing_price = listing_amenities = listing_property_type = listing_sold_date = listing_domain_insight = listing_schools = listing_url = 0
    
    #looping through listings in a page
    reset_point = driver.current_url
    
    try:
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div[4]/div[2]/div[1]/div[2]/ul/li[' + str(num_page) + ']/div/div[2]/div/a/h2/span[1]')))
        element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[4]/div[2]/div[1]/div[2]/ul/li[' + str(num_page) + ']/div/div[2]/div/a/h2/span[1]')
        element.click()

        #Grabs listing description
        try:
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//button[@data-testid="listing-details__description-button"]')))
            driver.find_element_by_xpath('//button[@data-testid="listing-details__description-button"]').send_keys(u'\ue007')
            WebDriverWait(driver, 1)
        except:
            pass
        listing_description = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[5]/div/div/div[3]/div[1]/div/div/div').text

        #Grabs listing's address
        element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[2]/div/div[1]')
        listing_address = element.text

        #Grabs listing's price
        element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[1]/div[1]/div')
        listing_price = element.text

        #Grabs listing's Amenities
        element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[3]/div')
        listing_amenities = element.text

        #Grabs listing's property type
        element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/div[4]/span')
        listing_property_type = element.text

        #Grabs listing's sold date
        element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[1]/div[1]/div/span[2]')
        listing_sold_date = element.text
        
        try:
            #Grabs listing's domain insight
            element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[5]/div/div/div[5]/div/div/p')
            listing_domain_insight = element.text

            #Grabs School details
            driver.find_element_by_xpath('//button[@data-testid="fe-co-school-catchment-view-more-link"]').send_keys(u'\ue007')
            element = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[9]/div/div/section/div[3]/ul')
            listing_schools = element.text
        except:
            pass

        #Grabs listing url
        listing_url = driver.current_url

        #Storing data
        data = [{'Suburb': suburb,'Address': listing_address, 'Description': listing_description, 'Price': listing_price, 'Amenities': listing_amenities, 'Type': listing_property_type, 'Date': listing_sold_date, 'Insights': listing_domain_insight, 'Schools': listing_schools, 'URL': listing_url}] 
        output = pd.DataFrame(data)    
        listings = listings.append(output, ignore_index=True)
        
    except:
        pass

    driver.get(reset_point)

In [8]:
listings.head()

,Suburb,Address,Description,Price,Amenities,Type,Date,Insights,Schools,URL
0,0,0,0,0,0,0,0,0,0,0
1,"21564 Properties sold in Abbotsford, VIC, 3067",,"Stylish living, spectacular views\nEnviable vi...","SOLD - $610,000",2\nBeds\n2\nBaths\n1\nParking,Apartment / Unit / Flat,Listing sold by advertiser 10th August 2020,"First listed on 20 July, this unit has been on...",Trinity Catholic School\n0.7 km away\nPrep-6Co...,https://www.domain.com.au/627-20-shamrock-stre...
2,"21564 Properties sold in Abbotsford, VIC, 3067",,THE ULTIMATE PROJECT BUT NOT HABITABLE NOW\n**...,"SOLD - $966,000",3\nBeds\n1\nBath\n−,House,Listing sold by advertiser 5th August 2020,"First listed on 23 July, this house has been o...",Abbotsford Primary School\n0.7 km away\nPrep-6...,https://www.domain.com.au/156-charles-street-a...
3,"21564 Properties sold in Abbotsford, VIC, 3067",,GENEROUS FAMILY HOME PARKS AND RIVER PRECINCT\...,"SOLD - $1,580,000",3\nBeds\n1\nBath\n2\nParking,House,Listing sold by advertiser 27th July 2020,"46 Marine Parade, Abbotsford VIC 3067 was last...","Lynall Hall Community School\n1.5 km away\nU, ...",https://www.domain.com.au/46-marine-parade-abb...
4,"21564 Properties sold in Abbotsford, VIC, 3067",,CITY SKYLINE VIEWS AWAIT\n**INSPECT STRICTLY B...,"SOLD - $1,092,500",3\nBeds\n2\nBaths\n2\nParking,Townhouse,Listing sold by advertiser 18th July 2020,"27/11 Nicholson Street, Abbotsford VIC 3067 wa...","Lynall Hall Community School\n1 km away\nU, 7-...",https://www.domain.com.au/27-11-nicholson-stre...


In [18]:
a = b = c = d = e = 0

In [19]:
listing_address = listing_description = listing_price = listing_amenities = listing_property_type = listing_sold_date = listing_domain_insight = listing_schools = listing_url = 0 

0